In [1]:
import pandas as pd
import numpy as np
import math
import statistics
from sklearn.datasets import load_digits, load_iris, load_boston, load_breast_cancer
from sklearn.model_selection import train_test_split
from graphviz import Digraph, Source, Graph
from multiprocessing import cpu_count, Pool
import sklearn
from IPython.display import Math
from sklearn.tree import export_graphviz
from copy import deepcopy

In [7]:
data = pd.read_csv('ML1/Tab.delimited.Cleaned.dataset.WITH.variable.labels.csv', sep = '\t', encoding = "ISO-8859-1")

/Users/adityavyas/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (17,55,59,61,65,68,69,70,83,90,91,92,93,120,121,122,123,126,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data.shape

(6344, 382)

In [9]:
data.head()

,session_id,session_date,last_update_date,session_last_update_date,referrer,creation_date,session_creation_date,expcomments,numparticipants_actual,numparticipants,...,diseaseforder,quoteorder,flagprimorder,sunkcostorder,anchorinorder,allowedforder,gamblerforder,moneypriorder,imaginedorder,iatorder
0,2400853,8/28/2013,8/28/13 12:15,8/28/13 12:15,abington,8/28/2013,8/28/2013,.,,5,...,4,3,7,2,8,6,10,1,11,12
1,2400856,8/28/2013,8/28/13 12:13,8/28/13 12:13,abington,8/28/2013,8/28/2013,.,,5,...,7,5,2,6,11,9,1,4,8,12
2,2400860,8/28/2013,8/28/13 12:15,8/28/13 12:15,abington,8/28/2013,8/28/2013,.,,5,...,2,8,9,3,5,10,6,4,11,12
3,2400868,8/28/2013,8/28/13 12:12,8/28/13 12:12,abington,8/28/2013,8/28/2013,.,,5,...,7,8,10,6,9,4,1,11,2,12
4,2400872,8/28/2013,8/28/13 12:11,8/28/13 12:11,abington,8/28/2013,8/28/2013,.,,.,...,10,3,11,6,9,5,4,1,2,12


# Identify Column Classes

In [10]:
mixed_columns = ['flagsupplement1', 'flagsupplement2', 'flagsupplement3', 'iatexplicitart1', 'iatexplicitart2',
                'iatexplicitart3', 'iatexplicitart4', 'iatexplicitart5', 'iatexplicitart6', 'iatexplicitmath1',
                'iatexplicitmath2', 'iatexplicitmath3', 'iatexplicitmath4', 'iatexplicitmath5', 'iatexplicitmath6',
                'sysjust1', 'sysjust2', 'sysjust3', 'sysjust4', 'sysjust5', 'sysjust6', 'sysjust7', 'sysjust8',
                'mturk.non.US', 'exprace']

all_NAN_columns = ['task_status', 'task_sequence', 'beginlocaltime']

numeric_columns = ['anchoring3ameter', 'anchoring3bmeter', 'anchoring1akm', 'anchoring1bkm', 'priorexposure8',
                    'mturk.duplicate', 'mturk.exclude.null', 'mturk.keep', 'mturk.exclude', 'gamblerfallacya_sd',
                    'gamblerfallacyb_sd', 'totexpmissed', 'numparticipants_actual', 'session_id', 'IATfilter',
                    'priorexposure3', 'priorexposure4', 'priorexposure5', 'priorexposure6', 'priorexposure7', 
                    'priorexposure9', 'priorexposure10', 'priorexposure11', 'priorexposure12', 'priorexposure13',
                    'numparticipants', 'age', 'anchoring1a', 'priorexposure1', 'priorexposure2', 'mturk.total.mini.exps',
                    'anchoring1b', 'anchoring2a', 'anchoring2b', 'anchoring3a', 'anchoring3b', 'anchoring4a',
                    'anchoring4b', 'artwarm', 'd_donotuse', 'ethnicity', 'flagdv1', 'flagdv2', 'flagdv3',
                    'flagdv4', 'flagdv5', 'flagdv6', 'flagdv7', 'flagdv8', 'gamblerfallacya', 'gamblerfallacyb',
                    'imaginedexplicit1', 'imaginedexplicit2', 'imaginedexplicit3', 'imaginedexplicit4',
                    'major', 'mathwarm', 'moneyagea', 'moneyageb', 'moneygendera', 'moneygenderb', 'omdimc3rt',
                    'omdimc3trt', 'quotea', 'quoteb', 'sunkcosta', 'sunkcostb', 'user_id', 'previous_session_id',
                    'order', 'meanlatency', 'meanerror', 'block2_meanerror', 'block3_meanerror', 'block5_meanerror',
                    'block6_meanerror', 'lat11', 'lat12', 'lat21', 'lat22', 'sd1', 'sd2', 'd_art1', 'd_art2', 'd_art',
                    'sunkDV', 'anchoring1', 'anchoring2', 'anchoring3', 'anchoring4', 'Ranchori', 'RAN001',
                    'RAN002', 'RAN003', 'Ranch1', 'Ranch2', 'Ranch3', 'Ranch4', 'gambfalDV', 'scalesreca', 'scalesrecb',
                    'reciprocityother', 'quotearec', 'quotebrec', 'quote', 'totalflagestimations', 'totalnoflagtimeestimations',
                    'flagdv', 'Sysjust', 'moneyfilter', 'Imagineddv', 'IATexpart', 'IATexpmath', 'IATexp.overall',
                    'IATEXPfilter', 'scalesorder', 'reciprocorder', 'diseaseforder', 'quoteorder', 'flagprimorder',
                    'sunkcostorder', 'anchorinorder', 'allowedforder', 'gamblerforder', 'moneypriorder', 'imaginedorder',
                    'iatorder']

date_columns = [col for col in data.columns if '_date' in col]

non_string_columns = mixed_columns + all_NAN_columns + numeric_columns + date_columns
string_columns = sorted([col for col in data.columns if col not in non_string_columns])

# Convert ['', ' ' and .] to NaN

In [11]:
data2 = data.copy()
data2 = data2.replace({'': np.nan, ' ': np.nan, '.': np.nan})

# Convert Columns to Respective Data Types

## 1. Numeric Columns

In [346]:
for col in numeric_columns:
    print(col)
    data2[col] = pd.to_numeric(data2[col])

anchoring3ameter
anchoring3bmeter
anchoring1akm
anchoring1bkm
priorexposure8
mturk.duplicate
mturk.exclude.null
mturk.keep
mturk.exclude
gamblerfallacya_sd
gamblerfallacyb_sd
totexpmissed
numparticipants_actual
session_id
IATfilter
priorexposure3
priorexposure4
priorexposure5
priorexposure6
priorexposure7
priorexposure9
priorexposure10
priorexposure11
priorexposure12
priorexposure13
numparticipants
age
anchoring1a
priorexposure1
priorexposure2
mturk.total.mini.exps
anchoring1b
anchoring2a
anchoring2b
anchoring3a
anchoring3b
anchoring4a
anchoring4b
artwarm
d_donotuse
ethnicity
flagdv1
flagdv2
flagdv3
flagdv4
flagdv5
flagdv6
flagdv7
flagdv8
gamblerfallacya
gamblerfallacyb
imaginedexplicit1
imaginedexplicit2
imaginedexplicit3
imaginedexplicit4
major
mathwarm
moneyagea
moneyageb
moneygendera
moneygenderb
omdimc3rt
omdimc3trt
quotea
quoteb
sunkcosta
sunkcostb
user_id
previous_session_id
order
meanlatency
meanerror
block2_meanerror
block3_meanerror
block5_meanerror
block6_meanerror
lat11
lat

## 2. Date Columns

In [347]:
for col in date_columns:
    print(col)
    data2[col] = pd.to_datetime(data2[col])

session_date
last_update_date
session_last_update_date
creation_date
session_creation_date
task_creation_date.0
task_creation_date.1
task_creation_date.2
task_creation_date.3
task_creation_date.4
task_creation_date.5
task_creation_date.6
task_creation_date.7
task_creation_date.8
task_creation_date.9
task_creation_date.10
task_creation_date.11
task_creation_date.12
task_creation_date.13
task_creation_date.14
task_creation_date.15
task_creation_date.16
task_creation_date.17
task_creation_date.18
task_creation_date.19
task_creation_date.20
task_creation_date.21
task_creation_date.22
task_creation_date.23
task_creation_date.24
task_creation_date.25
task_creation_date.26
task_creation_date.27
task_creation_date.28
task_creation_date.29
task_creation_date.30
task_creation_date.31
task_creation_date.32
task_creation_date.33
task_creation_date.34
task_creation_date.35
task_creation_date.36
task_creation_date.37
task_creation_date.38
task_creation_date.39
task_creation_date.40
task_creation_dat

## 3. String Columns

In [348]:
for col in string_columns:
    print(col)
    data2[col] = data2[col].astype(str)

ContactGroup
MoneyGroup
allowedforbidden
allowedforbiddenGroup
allowedforbiddena
allowedforbiddenb
anch1group
anch2group
anch3group
anch4group
citizenship
citizenship2
compensation
diseaseframinga
diseaseframingb
expcomments
expgender
exprunafter
exprunafter2
feedback
filter_$
flagGroup
flagfilter
flagtimeestimate1
flagtimeestimate2
flagtimeestimate3
flagtimeestimate4
gainlossDV
gainlossgroup
gambfalgroup
iat_exclude
imagineddescribe
imptaskto
lab_or_online
moneyethnicitya
moneyethnicityb
mturk.Submitted.PaymentReq
nativelang
nativelang2
noflagtimeestimate1
noflagtimeestimate2
noflagtimeestimate3
noflagtimeestimate4
o1
o10
o11
o2
o3
o4
o5
o6
o7
o8
o9
omdimc3
partgender
politicalid
previous_session_schema
quoteGroup
race
reciprocitygroup
reciprocityothera
reciprocityotherb
reciprocityus
reciprocityusa
reciprocityusb
recruitment
referrer
religion
sample
scales
scalesa
scalesb
scalesgroup
separatedornot
session_created_by
session_status
sex
study_name
study_url
sunkgroup
task_id.0
task_id

# Generating Synthetic Data

## Variational Autoencoder

In [316]:
class VariationalAutoencoder():
    
    SigmoidActivation = "sigmoid"
    ReLUActivation = "relu"
    LinearActivation = "linear"
    LeakyReLUActivation = "lrelu"
    
    def __init__(self, 
                 learning_rate = 0.04, 
                 batch_size = 32,
                 num_hidden_layers = None,
                 num_neurons_each_layer = None,
                 z_shape = 20,
                 epochs = 10):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.num_hidden_layers = num_hidden_layers
        self.num_neurons_each_layer = num_neurons_each_layer
        self.z_shape = z_shape
        
        self.activations_functions = {
            self.SigmoidActivation: self._sigmoid,
            self.LeakyReLUActivation: self._leaky_relu,
            self.ReLUActivation: self._relu,
            self.LinearActivation: self._linear
        }
        self.activations_derivatives = {
            self.SigmoidActivation: self._sigmoid_derivative,
            self.LeakyReLUActivation: self._leaky_relu_derivative,
            self.ReLUActivation: self._relu_derivative,
            self.LinearActivation: self._linear_derivative
        }
        
        # Activations for Encoder and Decoder
        self.encoder_activations = [self.LeakyReLUActivation] * self.num_hidden_layers + [self.LinearActivation]
        self.decoder_activations = [self.ReLUActivation] * self.num_hidden_layers + [self.SigmoidActivation]
        
        self.num_neurons_each_encoder_layer = self.num_neurons_each_layer
        self.num_neurons_each_decoder_layer = self.num_neurons_each_layer[::-1]
        

    def _sigmoid(self, x):
        x = np.select([x < 0, x >= 0], [1 - 1/(1 + np.exp(x)), 1/(1 + np.exp(-x))])
        return x

    def _relu(self, x):
        return np.maximum(0, x)
    
    def _leaky_relu(self, x):
        return np.maximum(0, x)

    def _linear(self, x):
        return x

    def _sigmoid_derivative(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def _relu_derivative(self, x):
        return (np.ones_like(x) * (x > 0))
    
    def _leaky_relu_derivative(self, x):
        return

    def _linear_derivative(self, x):
        return np.ones_like(x)
    
    def _binary_cross_entropy_loss(self, y_hat, y):
        loss = np.sum(-y * np.log(y_hat + 1e-15) - (1 - y) * np.log(1 - y_hat + 1e-15))
        return loss
    
    def _encoder(self, X):
        encoder_out = []
        
        for curr_layer in self.encoder_layers:
            encoder_out.append([])
                
            # Get the activation for this layer and its function
            activation_for_this_layer = self.encoder_activations[curr_layer]
            activation_function = self.activations_functions[activation_for_this_layer]

            if curr_layer == 0:
                previous_layer_output = X
            else:
                previous_layer_output = encoder_out[curr_layer - 1].copy()
                previous_layer_output = np.insert(previous_layer_output, obj = 0, values = 1, axis = 1)
            
            if curr_layer != self.encoder_layers[-1]:
                encoder_out[curr_layer] = activation_function(previous_layer_output @ self.encoder_weights[curr_layer].T)
            else:
                encoder_weights_last_layer = np.transpose(self.encoder_weights[curr_layer], axes = (0, 2, 1))
                encoder_out[curr_layer] = activation_function(previous_layer_output @ encoder_weights_last_layer)
        
        encoder_out = np.array(encoder_out)
        mu, log_sigma = encoder_out[-1][0], encoder_out[-1][1]
        return mu, log_sigma
    
    def _decoder(self, z):
        
        decoder_out = []
        
        for curr_layer in self.decoder_layers:
            decoder_out.append([])
                
            # Get the activation for this layer and its function
            activation_for_this_layer = self.decoder_activations[curr_layer]
            activation_function = self.activations_functions[activation_for_this_layer]

            if curr_layer == 0:
                previous_layer_output = z
            else:
                previous_layer_output = decoder_out[curr_layer - 1].copy()
            previous_layer_output = np.insert(previous_layer_output, obj = 0, values = 1, axis = 1)
            
            decoder_out[curr_layer] = activation_function(previous_layer_output @ self.decoder_weights[curr_layer].T)
        
        decoder_out = decoder_out[-1]
        return decoder_out
    
    def _forward(self, X):
        
        # Encode
        mu, log_sigma = self._encoder(X)
        
        # Reparametrization trick to sample z from gaussian. First sample from standar normal distribution.
        # Then we use z = mu + sigma*x to get our latent variable.
        self.rand_sample = np.random.standard_normal(size = (self.batch_size, self.z_shape))
        self.sample_z = mu + np.exp(log_sigma * .5) * self.rand_sample
        
        # Decode
        decoder_out = self._decoder(self.sample_z)
        
        return decoder_out, mu, log_sigma
    
    def _backward(self):
        return
    
    def _initialise_weights(self, input_shape):
        
        # Encoder Layers
        self.num_neurons_each_encoder_layer.append(2) # 2 for two outputs - mu and sigma
        self.total_encoder_layers = self.num_hidden_layers + 1 # +1 for the last output layer
        self.encoder_layers = range(self.total_encoder_layers)
        
        # Decoder Layers
        self.num_neurons_each_decoder_layer.append(input_shape) # Last layer of decoder has input shape
        self.total_decoder_layers = self.num_hidden_layers + 1 # +1 for the last output layer
        self.decoder_layers = range(self.total_decoder_layers)

        # Empty weight arrays
        self.encoder_weights = []
        self.decoder_weights = []

        # Initialise encoder weights
        for layer in self.encoder_layers:
            self.encoder_weights.append([])

            number_of_neurons_in_this_layer = self.num_neurons_each_encoder_layer[layer]
            if layer == 0:
                fan_in = input_shape
                fan_out = number_of_neurons_in_this_layer
                previous_layer_shape = fan_in
            else:
                fan_in = self.num_neurons_each_encoder_layer[layer - 1]
                fan_out = number_of_neurons_in_this_layer
                previous_layer_shape =  1 + fan_in
            
            if layer != self.encoder_layers[-1]:
                self.encoder_weights[layer] = np.random.uniform(low = -2.0/(fan_in + fan_out),
                                                                high = 2.0/(fan_in + fan_out),
                                                                size = (number_of_neurons_in_this_layer,
                                                                       previous_layer_shape))
            else:
                # Last layer of encoder outputs mu and sigma whose dimensions
                # are of shape z_shape.
                self.encoder_weights[layer] = np.random.uniform(low = -2.0/(fan_in + fan_out),
                                                                high = 2.0/(fan_in + fan_out),
                                                                size = (number_of_neurons_in_this_layer,
                                                                       self.z_shape,
                                                                       previous_layer_shape))
                
                
                
        # Initialise decoder weights
        for layer in self.decoder_layers:
            self.decoder_weights.append([])

            number_of_neurons_in_this_layer = self.num_neurons_each_decoder_layer[layer]
            if layer == 0:
                # Input to decoder is the latent variable constructed from
                # gaussian distribution
                fan_in = self.z_shape
            else:
                fan_in = self.num_neurons_each_layer[layer - 1]
            
            fan_out = number_of_neurons_in_this_layer
            previous_layer_shape = 1 + fan_in
            self.decoder_weights[layer] = np.random.uniform(low = -2.0/(fan_in + fan_out),
                                                            high = 2.0/(fan_in + fan_out),
                                                            size = (number_of_neurons_in_this_layer, 
                                                                   previous_layer_shape))

        self.encoder_weights = np.array(self.encoder_weights)
        self.decoder_weights = np.array(self.decoder_weights)
        self.old_encoder_weights = deepcopy(self.encoder_weights)
        self.old_decoder_weights = deepcopy(self.decoder_weights)
        
    def _get_batches(self, X):
        for i in range(0, X.shape[0], self.batch_size):
            yield X[i: i + self.batch_size]
    
    def fit(self, X):
        
        # Add a bias column to X
        X_new = np.column_stack((np.ones(len(X)), X))
        
        # Initialise weights using Glorot Uniform initialiser
        self._initialise_weights(X_new.shape[1])
        
        # Get batches
        batches = self._get_batches(X_new)
        
        iterations = 0
        while iterations <= self.epochs:
            
            # Train using mini-batch SGD
            for x_batch in batches:
                
                # Forward pass
                out, mu, log_sigma = self._forward(x_batch)
                
                # Reconstruction Loss

In [315]:
vae = VariationalAutoencoder(num_hidden_layers = 1, num_neurons_each_layer = [3], batch_size = 5)
vae.fit(X)

[[0.68266696 0.5713713  0.12659436 0.81292579 0.48890174 0.18363954
  0.57537288 0.49415602 0.40522787 0.47704483 0.23208769 0.43455381
  0.46747268 0.6954429  0.70064705 0.52701089 0.79557236 0.93616276
  0.80445233 0.10518495 0.64954404 0.17540005 0.17630946 0.3074999
  0.07983031 0.47142763 0.10381405 0.58969382 0.49564933 0.10087955
  0.30884341]
 [0.49961903 0.49099358 0.4969924  0.49646962 0.49949895 0.48845431
  0.49969168 0.4857558  0.50982183 0.50457645 0.49103865 0.49466304
  0.49511137 0.5060591  0.50848698 0.50872141 0.49063465 0.50457978
  0.49776493 0.50393182 0.49222342 0.4980785  0.49553691 0.48611342
  0.48675214 0.51377622 0.48889061 0.48897981 0.51020847 0.49058444
  0.48986356]
 [0.50465918 0.46782867 0.51486879 0.50802514 0.4943199  0.47005183
  0.51466763 0.49162345 0.5099383  0.4910997  0.48497425 0.49784574
  0.49781357 0.50465652 0.49005116 0.5176684  0.49520983 0.51290347
  0.48525044 0.51752813 0.49194769 0.49831259 0.48038102 0.47093954
  0.48180639 0.522018

NameError: name 's' is not defined

# ML Models

## Linear Regression

In [12]:
class LinearRegression():
    
    def __init__(self, bias = None, weights = None, fit_intercept = True):
        self.bias = bias
        self.weights = weights
        self.fit_intercept = fit_intercept
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)), X))
        else:
            X = np.column_stack((np.zeros(len(X)), X))
        self.all_weights = np.linalg.inv(np.dot(X.T, X)).dot(X.T).dot(y)
        self.weights = self.all_weights[1:]
        self.bias = self.all_weights[0]
    
    def predict(self, X):
        self.weights = self.weights.reshape(1, -1)
        predictions = self.bias + np.dot(self.weights, X.T)
        return predictions[0]
    
    def get_mse(self, y_true, y_pred):
        return np.sum((y_true - y_pred)**2)/y_true.shape[0]

## Ridge Regression

In [13]:
class RidgeRegression():
    
    def __init__(self, 
                 bias = None, 
                 weights = None, 
                 lambda_param = 10, 
                 fit_intercept = True):
        self.bias = bias
        self.weights = weights
        self.fit_intercept = fit_intercept
        self.lambda_param = lambda_param
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)), X))
        else:
            X = np.column_stack((np.zeros(len(X)), X))
            
        self.all_weights = np.linalg.inv(np.dot(X.T, X) + \
                            self.lambda_param * np.identity(X.shape[1])).dot(X.T).dot(y)
        self.weights = self.all_weights[1:]
        self.bias = self.all_weights[0]
    
    def predict(self, X):
        self.weights = self.weights.reshape(1, -1)
        predictions = self.bias + np.dot(self.weights, X.T)
        return predictions[0]
    
    def get_mse(self, y_true, y_pred):
        return np.sum((y_true - y_pred)**2)/y_true.shape[0]

## Lasso Regression

In [14]:
class LassoRegression():
    
    def __init__(self, 
                 bias = None, 
                 weights = None, 
                 lambda_param = 10, 
                 max_iters = 100, 
                 fit_intercept = True):
        self.bias = 0    
        self.lambda_param = lambda_param
        self.max_iters = max_iters
        self.fit_intercept = fit_intercept
    
    def _soft_threshold(self, x, lambda_):
        if x > 0.0 and lambda_ < abs(x):
            return x - lambda_
        elif x < 0.0 and lambda_ < abs(x):
            return x + lambda_
        else:
            return 0.0
    
    def fit(self, X, y):
        
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)), X))
        else:
            X = np.column_stack((np.zeros(len(X)), X))
            
        row_length, column_length = X.shape
        
        # Define the weights
        self.weights = np.zeros((1, column_length))[0]
        if self.fit_intercept:
            self.weights[0] = np.sum(y - \
                                np.dot(X[:, 1:], self.weights[1:]))/(X.shape[0])
        
        #Looping until max number of iterations
        for iteration in range(self.max_iters):
            start = 1 if self.fit_intercept else 0
            
            #Looping through each coordinate
            for j in range(start, column_length):
                    
                tmp_weights = self.weights.copy()
                tmp_weights[j] = 0.0
                r_j = y - np.dot(X, tmp_weights)
                arg1 = np.dot(X[:, j], r_j)
                arg2 = self.lambda_param * X.shape[0]

                self.weights[j] = self._soft_threshold(arg1, arg2)/(X[:, j]**2).sum()

                if self.fit_intercept:
                    self.weights[0] = np.sum(y - \
                                        np.dot(X[:, 1:], self.weights[1:]))/(X.shape[0])
        
        self.bias = self.weights[0]
        self.weights = self.weights[1:]
    
    def predict(self, X):
        self.weights = self.weights.reshape(1, -1)
        predictions = self.bias + np.dot(self.weights, X.T)
        return predictions[0]
    
    def get_mse(self, y_true, y_pred):
        return np.sum((y_true - y_pred)**2)/y_true.shape[0]

## Logistic Regression

In [8]:
class LogisticRegression():
    
    def __init__(self,  
                 weights = None, 
                 bias = None, 
                 fit_intercept = True,
                 decision_threshold = 0.5,
                 epochs = 50,
                 solver = 'sgd',
                 batch_size = 30,
                 learning_rate = 0.05,
                 tolerance = 1e-13):
        self.weights = weights
        self.bias = bias
        self.fit_intercept = fit_intercept
        self.tolerance = tolerance
        self.decision_threshold = decision_threshold
        self.epochs = epochs
        self.solver = solver
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        self.solver_func = {'newton': self._newton_solver,
                            'sgd': self._sgd_solver}
    
    def _sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))
        
    def _log_likelihood(self, X, y):
        P = self._sigmoid(X @ self.weights)
        P = P.reshape(-1, 1)
        log_P = np.log(P + 1e-16)

        P_ = 1 - P
        log_P_ = np.log(P_ + 1e-16)
        log_likelihood = np.sum(y*log_P + (1 - y)*log_P_)
        return log_likelihood
    
    def _get_true_class_labels(self, labels):
        true_labels = np.array([self.class_range_to_actual_classes[i] for i in labels])
        return true_labels
    
    def _get_batches(self, X, y):
        for i in range(0, X.shape[0], self.batch_size):
            yield (X[i: i + self.batch_size], y[i: i + self.batch_size])
            
    def _convert_y(self, y):
        self.actual_classes = sorted(np.unique(y))
        self.class_range = [0, 1]
        self.class_range_to_actual_classes = dict(zip(*(self.class_range, self.actual_classes)))
        self.actual_classes_to_class_range = dict(zip(*(self.actual_classes, self.class_range)))
        
        y_ = np.array([self.actual_classes_to_class_range[i] for i in y])
        y_ = y_.reshape(-1, 1)
        return y_
    
    def _newton_solver(self, X, y):
        log_likelihood = self._log_likelihood(X, y)
        iterations = 0
        delta = np.inf
        while(np.abs(delta) > self.tolerance and iterations < self.epochs):
            iterations += 1
            
            # Calculate positive class probabilities: p = sigmoid(W*x + b)
            z = X @ self.weights
            P = self._sigmoid(z)
            P = P.reshape(-1, 1)

            # First derivative of loss w.r.t weights
            grad = X.T @ (P - y)

            # Hessian of loss w.r.t weights
            P_ = 1 - P
            W = P * P_
            W = W.reshape(1, -1)[0]
            W = np.diag(W)
            hess = X.T @ W @ X

            # Weight update using Newton-Rhapson Method
            self.weights -= np.linalg.inv(hess) @ grad
            
            # Calculate new log likelihood
            new_log_likelihood = self._log_likelihood(X, y)
            delta = log_likelihood - new_log_likelihood
            log_likelihood = new_log_likelihood
    
    def _sgd_solver(self, X, y):
        iterations = 0
        while(iterations < self.epochs):
            iterations += 1
            
            # Get batches
            batches = self._get_batches(X, y)
            
            # Update weights using Mini batch stochastic gradient descent
            for (x_batch, y_batch) in batches:
                
                # Raw output
                z = x_batch @ self.weights
                
                # Calculate positive class probabilities: p = sigmoid(W*x + b)
                P = self._sigmoid(z)

                # First derivative of loss w.r.t weights
                grad = x_batch.T @ (P - y_batch)
                
                # Update weights
                self.weights -= self.learning_rate * grad
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)), X))
        else:
            X = np.column_stack((np.zeros(len(X)), X))
        row_length, column_length = X.shape
        
        # Define the weights
        self.weights = np.zeros((column_length, 1))
        
        # Convert y to {0, 1}
        y = self._convert_y(y)
        
        # Use the solver
        self.solver_func[self.solver](X, y)
        
    def predict_proba(self, X):
        
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)), X))
        else:
            X = np.column_stack((np.zeros(len(X)), X))
        
        z = X @ self.weights
        predicted_probs = self._sigmoid(z)
        return predicted_probs
    
    def predict(self, X):
        predict_probs = self.predict_proba(X)
        preds = np.where(predict_probs < 0.5, 0, 1).flatten()
        true_preds = self._get_true_class_labels(preds)
        return true_preds
    
    def get_accuracy(self, y, y_hat):
        return sum(y == y_hat)*100/len(y)

## Multiclass Logistic Regression

In [434]:
class MultiClassLogisticRegression():
    
    def __init__(self,  
                 weights = None, 
                 bias = None, 
                 fit_intercept = True,
                 epochs = 50,
                 learning_rate = 0.05,
                 batch_size = 50):
        self.weights = weights
        self.learning_rate = learning_rate
        self.bias = bias
        self.fit_intercept = fit_intercept
        self.epochs = epochs
        self.batch_size = batch_size
    
    def _softmax(self, z):
        
        # We only calculate the softmax probabilities of the first (K-1) classes
        z_ = z[:, :(z.shape[1] - 1)]
        e_x = np.exp(z_)
        out_k_minus_1 = e_x / (1 + e_x.sum(axis = 1, keepdims = True))
        
        # Probability for last K = 1 - p((K - 1))
        out_k = 1 - out_k_minus_1.sum(axis = 1)
        out = np.column_stack((out_k_minus_1, out_k))
        
        return out
    
    def _get_true_class_labels(self, P):
        labels = P.argmax(axis = 1)
        labels = np.array([self.class_range_to_actual_classes[i] for i in labels])
        return labels
    
    def _calculate_cross_entropy(self, y, log_yhat):
        return -np.sum(y * log_yhat, axis = 1)
    
    def _convert_to_indicator(self, y):
        y_indicator = np.zeros((y.shape[0], self.num_classes))
        for index, y_value in enumerate(y):
            class_range_mapping = self.actual_classes_to_class_range[y_value]
            y_indicator[index, class_range_mapping] = 1
        return y_indicator
    
    def _get_batches(self, X, y):
        for i in range(0, X.shape[0], self.batch_size):
            yield (X[i: i + self.batch_size], y[i: i + self.batch_size])
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)), X))
        else:
            X = np.column_stack((np.zeros(len(X)), X))
        row_length, column_length = X.shape
        
        # Number of unique classes
        self.actual_classes = sorted(np.unique(y))
        self.num_classes = len(self.actual_classes)
        
        # This will generate a list of [0,1,2,3....]. However, we want to map these class labels
        # to the original class labels in Y
        self.class_range = list(range(self.num_classes))
        self.actual_classes_to_class_range = {i: self.actual_classes[index] for index, i in enumerate(self.actual_classes)}
        self.class_range_to_actual_classes = {v: k for k, v in self.actual_classes_to_class_range.items()}
        
        # Convert y to indicator matrix form e.g. If y belongs to class 3, then y = [0,0,1,0..0]
        y = self._convert_to_indicator(y)
        
        # Define the weights, shape = (P + 1, K)
        self.weights = np.zeros((column_length, self.num_classes))
        
        iterations = 0
        while(iterations < self.epochs):
            iterations += 1
            
            # Get batches
            batches = self._get_batches(X, y)
            
            # Update weights using Mini batch stochastic gradient descent
            for (x_batch, y_batch) in batches:
                
                # Get raw output
                z = x_batch @ self.weights
                
                # Calculate class probabilities from raw output, shape = (B, K); B = batch size
                P = self._softmax(z)
                
                # Calculate gradient
                grad = x_batch.T @ (P - y_batch)
                
                # Update weights
                self.weights -= self.learning_rate * grad
                
    def predict_proba(self, X):
        
        if self.fit_intercept:
            X = np.column_stack((np.ones(len(X)), X))
        else:
            X = np.column_stack((np.zeros(len(X)), X))
        
        z = X @ self.weights
        predicted_probs = self._softmax(z)
        return predicted_probs
    
    def predict(self, X):
        predicted_probs = self.predict_proba(X)
        preds = self._get_true_class_labels(predicted_probs)
        return preds
    
    def get_accuracy(self, y, y_hat):
        return sum(y == y_hat)*100/len(y)

In [377]:
iris = load_iris()
X_iris, y_iris = iris.data, iris.target

In [435]:
multi_logreg = MultiClassLogisticRegression(epochs = 100, batch_size = 60, learning_rate = 0.005)
multi_logreg.fit(X_iris, y_iris)

In [436]:
multi_logreg.get_accuracy(preds, y_iris)

97.33333333333333

## Perceptrons

In [551]:
class Perceptron():
    
    def __init__(self, w = None, b = None, termination_steps = 0):
        self.w = w
        self.b = b
        self.termination_steps = termination_steps
        self.prev_weights = []
        self.prev_bias = []
    
    def fit(self, X, y):
        self.w = np.zeros((1, X.shape[1]))
        self.b = 0
        
        while(True):
            
            # Update w, b by iterating over each row of X
            misclassified_count = 0
            for index in range(len(y)):
                
                # Update when a data point gets misclassified
                pred = np.sign(np.dot(self.w, X[index]) + self.b)[0]
                if pred != y[index]:
                    misclassified_count += 1
                    self.w += X[index]*y[index]
                    self.b += y[index]
            
            # Termination condition
            if (misclassified_count == 0) or (misclassified_count >= 0.3*len(y) \
                                              and self.termination_steps >= 1e5):
                break
                
            self.prev_weights.append(self.w.copy())
            self.prev_bias.append(copy(self.b))
            self.termination_steps += 1
    
    def predict(self, X):
        return np.sign(self.w @ X.T + self.b)[0]
    
    def get_accuracy(self, y, y_hat):
        return sum(y == y_hat)*100/len(y)

## Decision Tree Classifier

In [18]:
class Node():
    def __init__(self, 
                 data = None, 
                 split_variable = None,
                 split_variable_value = None,
                 left = None, 
                 right = None, 
                 depth = 0,
                 criterion_value = None):
        self.data = data
        self.split_variable = split_variable
        self.split_variable_value = split_variable_value
        self.left = left
        self.right = right
        self.criterion_value = criterion_value
        self.depth = depth

In [606]:
class DecisionTreeClassifier():
    def __init__(self,
                 root = None, 
                 criterion = "gini", 
                 max_depth = None,
                 significance = None,
                 significance_threshold = 3.841,
                 min_samples_split = 1):
        self.root = root
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.significance = significance
        self.significance_threshold = significance_threshold
        
        self.split_score_funcs = {'gini': self._calculate_gini_values,
                                  'entropy': self._calculate_entropy_values}
    
    def _get_proportions(self, X):
        counts_of_classes_of_y = X['Y'].value_counts()
        proportions_of_classes_of_y = counts_of_classes_of_y/X.shape[0]
        return proportions_of_classes_of_y
    
    def _get_entropy_index(self, X):
        return
    
    def _calculate_entropy_values(self, X, feature):
        return
    
    def _get_gini_index(self, X):
        if X.empty:
            return 0
        
        # Get proportion of all classes of y in X
        proportions = self._get_proportions(X)
        
        # Calculate the gini index
        gini_index = 1 - np.sum(proportions**2)
        return gini_index
    
    def _calculate_gini_values(self, X, feature):
        
        # Calculate unique values of X. For a feature, there are different
        # values on which that feature can be split
        classes = X[feature].unique()
        
        # Calculate the gini value for a split on each unique value of the feature.
        best_gini_score = 999
        best_feature_value = ""
        for unique_value in classes:
            # Split data
            left_split = X[X[feature] <= unique_value]
            right_split = X[X[feature] > unique_value]
            
            # Get gini scores of left, right nodes
            gini_value_left_split = self._get_gini_index(left_split)
            gini_value_right_split = self._get_gini_index(right_split)
            
            # Combine the 2 scores to get the overall score for the split
            gini_score_of_current_value = (left_split.shape[0]/X.shape[0]) * gini_value_left_split + \
                                           (right_split.shape[0]/X.shape[0]) * gini_value_right_split
            
            if gini_score_of_current_value < best_gini_score:
                best_gini_score = gini_score_of_current_value
                best_feature_value = unique_value
        
        return best_gini_score, best_feature_value
    
    def _get_best_split_feature(self, X):
        best_split_score = 999
        best_feature = ""
        best_value = None
        columns = X.drop('Y', 1).columns
        
        for feature in columns:
            
            # Calculate the best split score and the best value
            # for the current feature.
            split_score, feature_value = self.split_score_funcs[self.criterion](X, feature)
            
            # Compare this feature's split score with the current best score
            if split_score < best_split_score:
                best_split_score = split_score
                best_feature = feature
                best_value = feature_value
        
        return best_feature, best_value, best_split_score
    
    def _split_data(self, X, X_depth = None):
        
        # Return if dataframe is empty, depth exceeds maximum depth or sample size exceeds
        # minimum sample size required to split.
        if X.empty or len(X['Y'].value_counts()) == 1 or X_depth == self.max_depth \
                            or X.shape[0] <= self.min_samples_split:
            return None, None, "", "", 0
        
        # Calculate the best feature to split X
        best_feature, best_value, best_score = self._get_best_split_feature(X)
        
        if best_feature == "":
            return None, None, "", "", 0
        
        # Create left and right nodes
        X_left = Node(data = X[X[best_feature] <= best_value].drop(best_feature, 1),
                      depth = X_depth + 1)
        X_right = Node(data = X[X[best_feature] > best_value].drop(best_feature, 1),
                       depth = X_depth + 1)

        return X_left, X_right, best_feature, best_value, best_score
    
    def _fit(self, X):
        
        # Handle the initial case
        if not (type(X) == Node):
            X = Node(data = X)
            self.root = X
        
        # Get the splits
        X_left, X_right, best_feature, best_value, best_score = self._split_data(X.data, X.depth)
        
        # Assign attributes of node X
        X.left = X_left
        X.right = X_right
        X.split_variable = best_feature
        X.split_variable_value = round(best_value, 3) if type(best_value) != str else best_value
        X.criterion_value = round(best_score, 3)
        
        # Return if no best variable found to split on. 
        # This means you have reached the leaf node.
        if best_feature == "":
            return
        
        # Recurse for left and right children
        self._fit(X_left)
        self._fit(X_right)
        
    def fit(self, X, y):
        # Combine the 2 and fit
        X = pd.DataFrame(X)
        X['Y'] = y
        self._fit(X)
    
    def predict(self, X):
        X = pd.DataFrame(X)
        X['Y'] = np.nan
        
        for index, row in X.iterrows():
            curr_node = self.root
            while(curr_node.left != None and curr_node.right != None):
                split_variable = curr_node.split_variable
                split_variable_value = curr_node.split_variable_value
                if X.loc[index, split_variable] <= split_variable_value:
                    curr_node = curr_node.left
                else:
                    curr_node = curr_node.right
            
            # Assign Y value
            X.loc[index, 'Y'] = max(curr_node.data['Y'].values, key = list(curr_node.data['Y'].values).count)    
        
        preds = X['Y'].values
        return preds
    
    def display_tree_structure(self):
        tree = Digraph('DecisionTree', 
                       filename = 'tree.dot', 
                       node_attr = {'shape': 'box'})
        tree.attr(size = '10, 20')
        root = self.root
        id = 0
        
        # queue with nodes to process
        nodes = [(None, root, 'root')]
        while nodes:
            parent, node, x = nodes.pop(0)
            
            # Generate appropriate labels for the nodes
            value_counts_length = len(node.data['Y'].value_counts())
            if node.split_variable != "":
                split_variable = node.split_variable
                split_variable_value = node.split_variable_value
            else:
                split_variable = "None"
                
            if value_counts_length > 1:
                label = str(split_variable) + '\n' + self.criterion + " = " + str(split_variable_value)
            else:
                label = "None"
            
            # Make edges between the nodes
            tree.node(name = str(id), 
                      label = label, 
                      color = 'black', 
                      fillcolor = 'goldenrod2', 
                      style = 'filled')
            
            if parent is not None:
                if x == 'left':
                    tree.edge(parent, str(id), color = 'sienna', 
                              style = 'filled', label = '<=' + ' ' + str(split_variable_value))
                else:
                    tree.edge(parent, str(id), color = 'sienna', 
                              style = 'filled', label = '>' + ' ' + str(split_variable_value))

            if node.left is not None:
                nodes.append((str(id), node.left, 'left'))

            if node.right is not None:
                nodes.append((str(id), node.right, 'right'))
            id += 1
            
        return tree
        
    def get_accuracy(self, y, y_hat):
        return sum(y == y_hat)*100/len(y)

## Decision Tree Regressor

In [617]:
class DecisionTreeRegressor():
    def __init__(self,
                 root = None, 
                 criterion = "mse", 
                 max_depth = None,
                 significance = None,
                 significance_threshold = 3.841,
                 min_samples_split = 1):
        self.root = root
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.significance = significance
        self.significance_threshold = significance_threshold
        
        self.split_score_funcs = {'mse': self._calculate_mse_values}
    
    def _get_mse(self, X):
        if X.empty:
            return 0
        
        # Calculate the mean square error with respect to the mean
        y = X['Y']
        y_mean = np.mean(y)
        mse = np.mean((y - y_mean)**2)
        return mse
    
    def _calculate_mse_values(self, X, feature):
        
        # Calculate unique values of X. For a feature, there are different
        # values on which that feature can be split
        classes = X[feature].unique()
        
        # Calculate the gini value for a split on each unique value of the feature.
        best_mse_score = 999
        best_feature_value = ""
        for unique_value in classes:
            # Split data
            left_split = X[X[feature] <= unique_value]
            right_split = X[X[feature] > unique_value]
            
            # Get gini scores of left, right nodes
            mse_value_left_split = self._get_mse(left_split)
            mse_value_right_split = self._get_mse(right_split)
            
            # Combine the 2 scores to get the overall score for the split
            mse_score_of_current_value = (left_split.shape[0]/X.shape[0]) * mse_value_left_split + \
                                           (right_split.shape[0]/X.shape[0]) * mse_value_right_split
            
            if mse_score_of_current_value < best_mse_score:
                best_mse_score = mse_score_of_current_value
                best_feature_value = unique_value
            
        return best_mse_score, best_feature_value
    
    def _get_best_split_feature(self, X):
        best_split_score = 999
        best_feature = ""
        best_value = None
        columns = X.drop('Y', 1).columns
        
        for feature in columns:
            
            # Calculate the best split score and the best value
            # for the current feature.
            split_score, feature_value = self.split_score_funcs[self.criterion](X, feature)
            
            # Compare this feature's split score with the current best score
            if split_score < best_split_score:
                best_split_score = split_score
                best_feature = feature
                best_value = feature_value
        
        return best_feature, best_value, best_split_score
    
    def _split_data(self, X, X_depth = None):
        
        # Return if dataframe is empty, depth exceeds maximum depth or sample size exceeds
        # minimum sample size required to split.
        if X.empty or len(X['Y'].value_counts()) == 1 or X_depth == self.max_depth \
                            or X.shape[0] <= self.min_samples_split:
            return None, None, "", "", 0
        
        # Calculate the best feature to split X
        best_feature, best_value, best_score = self._get_best_split_feature(X)
        
        if best_feature == "":
            return None, None, "", "", 0
        
        # Create left and right nodes
        X_left = Node(data = X[X[best_feature] <= best_value].drop(best_feature, 1),
                      depth = X_depth + 1)
        X_right = Node(data = X[X[best_feature] > best_value].drop(best_feature, 1),
                       depth = X_depth + 1)

        return X_left, X_right, best_feature, best_value, best_score
    
    def _fit(self, X):
        
        # Handle the initial case
        if not (type(X) == Node):
            X = Node(data = X)
            self.root = X
        
        # Get the splits
        X_left, X_right, best_feature, best_value, best_score = self._split_data(X.data, X.depth)
        
        # Assign attributes of node X
        X.left = X_left
        X.right = X_right
        X.split_variable = best_feature
        X.split_variable_value = round(best_value, 3) if type(best_value) != str else best_value
        X.criterion_value = round(best_score, 3)
        
        # Return if no best variable found to split on. 
        # This means you have reached the leaf node.
        if best_feature == "":
            return
        
        # Recurse for left and right children
        self._fit(X_left)
        self._fit(X_right)
        
    def fit(self, X, y):
        
        # Combine the 2 and fit
        X = pd.DataFrame(X)
        X['Y'] = y
        self._fit(X)
    
    def predict(self, X):
        X = pd.DataFrame(X)
        X['Y'] = np.nan
        
        preds = []
        for index, row in X.iterrows():
            curr_node = self.root
            while(curr_node.left != None and curr_node.right != None):
                split_variable = curr_node.split_variable
                split_variable_value = curr_node.split_variable_value
                if X.loc[index, split_variable] <= split_variable_value:
                    curr_node = curr_node.left
                else:
                    curr_node = curr_node.right
            
            # Get prediction
            preds.append(np.mean(curr_node.data['Y'].values))
        
        preds = X['Y'].values
        return preds
    
    def display_tree_structure(self):
        tree = Digraph('DecisionTree', 
                       filename = 'tree.dot', 
                       node_attr = {'shape': 'box'})
        tree.attr(size = '10, 20')
        root = self.root
        id = 0
        
        # queue with nodes to process
        nodes = [(None, root, 'root')]
        while nodes:
            parent, node, x = nodes.pop(0)
            
            # Generate appropriate labels for the nodes
            value_counts_length = len(node.data['Y'].value_counts())
            if node.split_variable != "":
                split_variable = node.split_variable
                split_variable_value = node.split_variable_value
            else:
                split_variable = "None"
                
            if value_counts_length > 1:
                label = split_variable + '\n' + self.criterion + " = " + \
                            str(node.criterion_value)
            else:
                label = "None"
            
            # Make edges between the nodes
            tree.node(name = str(id), 
                      label = label, 
                      color = 'black', 
                      fillcolor = 'goldenrod2', 
                      style = 'filled')
            
            if parent is not None:
                if x == 'left':
                    tree.edge(parent, str(id), color = 'sienna', 
                              style = 'filled', label = '<=' + ' ' + str(split_variable_value))
                else:
                    tree.edge(parent, str(id), color = 'sienna', 
                              style = 'filled', label = '>' + ' ' + str(split_variable_value))

            if node.left is not None:
                nodes.append((str(id), node.left, 'left'))

            if node.right is not None:
                nodes.append((str(id), node.right, 'right'))
            id += 1
            
        return tree
        
    def get_error(self, y, y_hat):
        return np.mean((y - y_hat)**2)

## Adaboost Binary Classifier

In [11]:
class AdaboostClassifier():
    
    def __init__(self, n_estimators = 100, weights = None):
        self.n_estimators = n_estimators
        self.weights = weights
        self.alphas = []
        
    def _convert_y(self, y):
        self.actual_classes = sorted(np.unique(y))
        self.class_range = [-1, 1]
        self.class_range_to_actual_classes = dict(zip(*(self.class_range, self.actual_classes)))
        self.actual_classes_to_class_range = dict(zip(*(self.actual_classes, self.class_range)))
        
        y_ = np.array([self.actual_classes_to_class_range[i] for i in y])
        return y_
    
    def _get_true_class_labels(self, labels):
        true_labels = np.array([self.class_range_to_actual_classes[i] for i in labels])
        return true_labels
    
    def fit(self, X, y):
        
        # Convert y to {-1, 1}
        y = self._convert_y(y)
        
        # Initialise weights for all data points
        row_length = X.shape[0]
        self.weights = np.ones((self.n_estimators, row_length))
        self.alphas = np.zeros((self.n_estimators, 1))
        self.estimators = np.empty((self.n_estimators, 1), dtype = object)
        
        time_step = 0
        for time_step in range(self.n_estimators):
            
            # Use a weak classifier to fit on data
            weak_classifier = LogisticRegression(solver = "sgd", epochs = 20)
            weak_classifier.fit(X, y)
            pred = weak_classifier.predict(X)
            
            # Get weighted error 
            weighted_sample_err = (np.sum((pred != y) * self.weights))/np.sum(self.weights)
            
            # Alpha for current classifer
            alpha_t = 1/2*np.log(((1 - weighted_sample_err)/weighted_sample_err) + 1e-16)
            self.alphas[time_step] = alpha_t
            self.estimators[time_step] = weak_classifier
            
            # Update weights of next time step for all data points
            if time_step == (self.n_estimators - 1):
                break
            self.weights[time_step + 1, :] = self.weights[time_step, :] * np.exp(-y * alpha_t * pred)
    
    def predict(self, X):
        preds = []
        self.estimators = self.estimators.flatten()
        self.alphas = self.alphas.flatten()
        for index in range(self.n_estimators):
            preds.append(self.alphas[index] * self.estimators[index].predict(X))
        
        preds = np.sum(preds, 0)
        preds = np.sign(preds)
        true_preds = self._get_true_class_labels(preds)
        return true_preds
    
    def get_accuracy(self, y, y_hat):
        return sum(y == y_hat)*100/len(y)

## Gradient Boost

## KNN

In [469]:
class KNeighbours():
    def __init__(self, k = 5, distance_metric = 'euclid', problem = "classify"):
        self.k = k
        self.distance_metric = distance_metric
        self.problem = problem
        self.prediction_functions = {'classify': self._top_k_votes,
                                     'regress': self._top_k_mean}
        self.eval_functions = {'classify': self._get_accuracy,
                               'regress': self._get_mse}
    
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def _euclidien_distance(self, x):
        return np.sqrt(np.sum((x - self.X)**2, axis = 1))
    
    def _top_k_mean(self, top_k):
        return np.mean(top_k)
    
    def _top_k_votes(self, top_k):
        return max(top_k, key = list(top_k).count)
    
    def _get_accuracy(self, pred, y):
        return np.sum((pred == y))*100/len(y)
    
    def _get_mse(self, pred, y):
        return np.mean((pred - y)**2)
    
    def predict(self, X):

        preds = list()
        for x in X:
            distances = self._euclidien_distance(x)
            
            # Zip the distances and y values together
            distances = zip(*(distances, self.y))
            
            # Sort the distances list by distance values in descending order
            distances = sorted(distances, key = lambda x: x[0])
            
            # Select top k distances
            top_k = distances[:(self.k)]
            
            top_k = np.array(top_k)
            top_k = top_k[:, 1]
            
            # Calculate mean of y values of these top k data items
            pred = self.prediction_functions[self.problem](top_k)
            preds.append(pred)
        
        return preds
    
    def evaluate(self, pred, y):
        eval_func = self.eval_functions[self.problem]
        return eval_func(pred, y)

## Neural Network Classifier

In [18]:
class NeuralNetworkClassifier():

    SigmoidActivation = "sigmoid"
    ReLUActivation = "relu"
    LinearActivation = "linear"

    def __init__(self,
                 num_hidden_layers = 2,
                 learning_rate = 0.1,
                 num_neurons_each_layer = None,
                 num_neurons_last_layer = 1,
                 batch_size = 32,
                 epochs = 10,
                 weights = None):
        self.weights = weights
        self.num_hidden_layers = num_hidden_layers
        self.num_neurons_each_layer = num_neurons_each_layer
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.num_neurons_last_layer = num_neurons_last_layer

        # Sigmoid activation for other layers. Linear activation for last layer
        self.activations = [self.SigmoidActivation] * self.num_hidden_layers + [self.SigmoidActivation]
        self.activations_functions = {
            self.SigmoidActivation: self._sigmoid,
            self.ReLUActivation: self._relu,
            self.LinearActivation: self._linear
        }
        self.activations_derivatives = {
            self.SigmoidActivation: self._sigmoid_derivative,
            self.ReLUActivation: self._relu_derivative,
            self.LinearActivation: self._linear_derivative
        }

    def _sigmoid(self, x):
        def sigfunc(x):
            if x < 0:
                return 1 - 1 / (1 + math.exp(x))
            else:
                return 1 / (1 + math.exp(-x))
        x_ = np.array([sigfunc(i) for i in x])
        return x_

    def _relu(self, x):
        return np.maximum(0, x)

    def _linear(self, x):
        return x

    def _sigmoid_derivative(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def _relu_derivative(self, x):
        return (np.ones_like(x) * (x > 0))

    def _linear_derivative(self, x):
        return np.ones_like(x)

    def _mse_loss(self, pred, y):
        return np.mean((pred - y) ** 2)

    def _initialise_weights(self, input_shape):

        self.num_neurons_each_layer.append(self.num_neurons_last_layer)
        self.total_layers = self.num_hidden_layers + 1
        self.layers = range(self.total_layers)

        # Initialising a numpy array of
        # shape = (number of hidden layers, number of neurons, number of weights per neuron) to store weights
        self.weights = []

        # Iterate through the layers
        for layer in self.layers:
            self.weights.append([])

            number_of_neurons_in_this_layer = self.num_neurons_each_layer[layer]
            if layer == 0:
                self.weights[layer] = np.random.normal(loc = 0,
                                                       scale = 0.5,
                                                       size = (number_of_neurons_in_this_layer, input_shape))
            else:
                # Adding 1 for the bias neuron
                self.weights[layer] = np.random.normal(loc = 0,
                                                       scale = 0.5,
                                                       size = (number_of_neurons_in_this_layer,
                                                     1 + self.num_neurons_each_layer[layer - 1]))

        self.weights = np.array(self.weights)
        self.old_weights = deepcopy(self.weights)

    def _update_weights(self):
        avg_batch_weight_derivatives = np.mean(self.batch_weight_derivatives, axis = 0)
        self.weights = self.old_weights - self.learning_rate * avg_batch_weight_derivatives
        self.old_weights = deepcopy(self.weights)
        self.batch_weight_derivatives = []

    def _backward(self, x, y, out):

        # The derivatives array will have the same shape as weights array. - one derivative for each
        # weight
        output_derivatives = deepcopy(out)
        weight_derivatives = deepcopy(self.weights)

        # Compute the output derivatives
        layers_reversed = self.layers[::-1]
        for curr_layer in layers_reversed:
            next_layer = curr_layer + 1

            # For the last layer simply use the formula
            if curr_layer == self.total_layers - 1:
                output_derivatives[curr_layer] = 2*(out[curr_layer] - y)
                continue

            # Get the activation derivative function for next layer
            activation_for_next_layer = self.activations[next_layer]
            activation_derivative = self.activations_derivatives[activation_for_next_layer]

            # The next layer output derivatives
            next_layer_output_derivatives = output_derivatives[next_layer]

            # Calculate the activation derivative. Add a 1 for the bias weight
            current_layer_output = out[curr_layer].copy()
            current_layer_output = np.insert(current_layer_output, obj = 0, values = 1)
            next_layer_activation_derivatives = activation_derivative(self.old_weights[next_layer] @ current_layer_output)
            next_layer_activation_derivatives = next_layer_activation_derivatives.reshape(-1, 1)

            # Remove the bias from the weights.
            next_layer_weights_without_bias = self.old_weights[next_layer][:, 1:]

            # Multiply each neuron's activation derivative with its weights. This is the Hadmard product
            second_term = next_layer_activation_derivatives * next_layer_weights_without_bias

            # Sum over all the neurons in the next layer to get the output derivative for each
            # neuron in the current layer. This is because each neuron contributes to all the neurons
            # in the next layer.
            output_derivatives[curr_layer] = next_layer_output_derivatives @ second_term

        # Update the weights using the output derivative calculated above
        for curr_layer in layers_reversed:

            # Get the activation for this layer and its derivative function
            activation_for_this_layer = self.activations[curr_layer]
            activation_derivative = self.activations_derivatives[activation_for_this_layer]

            # If first layer then use the data as the previous layer.
            if curr_layer == 0:
                previous_layer_output = x
            else:
                prev_layer = curr_layer - 1
                previous_layer_output = out[prev_layer].copy()
                previous_layer_output = np.insert(previous_layer_output, obj = 0, values = 1)

            # Current layer output derivatives
            curr_layer_output_derivatives = output_derivatives[curr_layer].reshape(-1, 1)

            # Get current layer's activation derivatives
            curr_layer_activation_derivatives = activation_derivative(self.old_weights[curr_layer] @ previous_layer_output)
            curr_layer_activation_derivatives = curr_layer_activation_derivatives.reshape(-1, 1)

            # For the current layer multiply each neuron's activation derivatives with all previous layer outputs.
            curr_layer_weight_derivatives = curr_layer_output_derivatives * \
                                            curr_layer_activation_derivatives * previous_layer_output
            weight_derivatives[curr_layer] = curr_layer_weight_derivatives

        # Append the current data point's weight derivatives in the batch derivatives array
        self.batch_weight_derivatives.append(weight_derivatives)

    def _forward(self, x):
        out = []
        for curr_layer in self.layers:
            out.append([])

            # Get the activation for this layer and its function
            activation_for_this_layer = self.activations[curr_layer]
            activation_function = self.activations_functions[activation_for_this_layer]

            if curr_layer == 0:
                previous_layer_output = x
            else:
                previous_layer_output = out[curr_layer - 1].copy()
                previous_layer_output = np.insert(previous_layer_output, obj = 0, values = 1)

            out[curr_layer] = activation_function(self.weights[curr_layer] @ previous_layer_output)

        out = np.array(out)
        return out

    def fit(self, X, y):

        # Add a bias column to X
        X_new = np.column_stack((np.ones(len(X)), X))

        # Initialise the weights of the network
        self._initialise_weights(X_new.shape[1])

        for epoch in range(self.epochs):

            # Initialise arrays to store all weight derivatives of the batch
            self.batch_weight_derivatives = []

            # Update weights using mini-batch stochastic gradient descent
            for data_index in range(X_new.shape[0]):
                out = self._forward(X_new[data_index])
                self._backward(X_new[data_index], y[data_index], out)

                if (data_index + 1) % self.batch_size == 0:
                    self._update_weights()

            predictions = self.predict(X)
            loss = self._mse_loss(predictions, y)
            print("Epoch = ", str(epoch + 1), " - ", "Loss = ", str(loss))

    def predict(self, X):

        # Add a bias column to X
        X_new = np.column_stack((np.ones(len(X)), X))

        preds = []
        for x in X_new:
            last_layer_out = self._forward(x)[-1]
            preds.append(np.argmax(last_layer_out))

        preds = np.array(preds)
        return preds

## Neural Network Regressor

In [328]:
class NeuralNetworkRegressor():

    SigmoidActivation = "sigmoid"
    ReLUActivation = "relu"
    LinearActivation = "linear"

    def __init__(self,
                 num_hidden_layers = 2,
                 learning_rate = 0.1,
                 num_neurons_each_layer = None,
                 num_neurons_last_layer = 1,
                 batch_size = 32,
                 epochs = 10,
                 weights = None):
        self.weights = weights
        self.num_hidden_layers = num_hidden_layers
        self.num_neurons_each_layer = num_neurons_each_layer
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.num_neurons_last_layer = num_neurons_last_layer

        # Sigmoid activation for other layers. Linear activation for last layer
        self.activations = [self.ReLUActivation] * self.num_hidden_layers + [self.LinearActivation]
        self.activations_functions = {
            self.SigmoidActivation: self._sigmoid,
            self.ReLUActivation: self._relu,
            self.LinearActivation: self._linear
        }
        self.activations_derivatives = {
            self.SigmoidActivation: self._sigmoid_derivative,
            self.ReLUActivation: self._relu_derivative,
            self.LinearActivation: self._linear_derivative
        }

    def _sigmoid(self, x):
        def sigfunc(x):
            if x < 0:
                return 1 - 1 / (1 + math.exp(x))
            else:
                return 1 / (1 + math.exp(-x))
        x_ = np.array([sigfunc(i) for i in x])
        return x_

    def _relu(self, x):
        return np.maximum(0, x)

    def _linear(self, x):
        return x

    def _sigmoid_derivative(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def _relu_derivative(self, x):
        return (np.ones_like(x) * (x > 0))

    def _linear_derivative(self, x):
        return np.ones_like(x)

    def _mse_loss(self, pred, y):
        return np.mean((pred - y) ** 2)

    def _initialise_weights(self, input_shape):

        self.num_neurons_each_layer.append(self.num_neurons_last_layer)
        self.total_layers = self.num_hidden_layers + 1
        self.layers = range(self.total_layers)

        # Initialising a numpy array of
        # shape = (number of hidden layers, number of neurons, number of weights per neuron) to store weights
        self.weights = []

        # Iterate through the layers
        for layer in self.layers:
            self.weights.append([])
            
            number_of_neurons_in_this_layer = self.num_neurons_each_layer[layer]
            if layer == 0:
                fan_in = input_shape
                fan_out = number_of_neurons_in_this_layer
                previous_layer_shape = fan_in
            else:
                fan_in = self.num_neurons_each_layer[layer - 1]
                fan_out = number_of_neurons_in_this_layer
                previous_layer_shape =  1 + fan_in

            self.weights[layer] = np.random.uniform(low = -2./(fan_in + fan_out),
                                                    high = 2./(fan_in + fan_out),
                                                    size = (number_of_neurons_in_this_layer, 
                                                           previous_layer_shape))

        self.weights = np.array(self.weights)
        self.old_weights = deepcopy(self.weights)

    def _update_weights(self):
        avg_batch_weight_derivatives = np.mean(self.batch_weight_derivatives, axis = 0)
        self.weights = self.old_weights - self.learning_rate * avg_batch_weight_derivatives
        self.old_weights = deepcopy(self.weights)
        self.batch_weight_derivatives = []

    def _backward(self, x, y, out):

        # The derivatives array will have the same shape as weights array. - one derivative for each
        # weight
        output_derivatives = deepcopy(out)
        weight_derivatives = deepcopy(self.weights)

        # Compute the output derivatives
        layers_reversed = self.layers[::-1]
        for curr_layer in layers_reversed:
            next_layer = curr_layer + 1

            # For the last layer simply use the formula
            if curr_layer == self.total_layers - 1:
                output_derivatives[curr_layer] = 2*(out[curr_layer] - y)
                continue

            # Get the activation derivative function for next layer
            activation_for_next_layer = self.activations[next_layer]
            activation_derivative = self.activations_derivatives[activation_for_next_layer]

            # The next layer output derivatives
            next_layer_output_derivatives = output_derivatives[next_layer]

            # Calculate the activation derivative. Add a 1 for the bias weight
            current_layer_output = out[curr_layer].copy()
            current_layer_output = np.insert(current_layer_output, obj = 0, values = 1)
            next_layer_activation_derivatives = activation_derivative(self.old_weights[next_layer] @ current_layer_output)
            next_layer_activation_derivatives = next_layer_activation_derivatives.reshape(-1, 1)

            # Remove the bias from the weights.
            next_layer_weights_without_bias = self.old_weights[next_layer][:, 1:]

            # Multiply each neuron's activation derivative with its weights. This is the Hadmard product
            second_term = next_layer_activation_derivatives * next_layer_weights_without_bias

            # Sum over all the neurons in the next layer to get the output derivative for each
            # neuron in the current layer. This is because each neuron contributes to all the neurons
            # in the next layer.
            output_derivatives[curr_layer] = next_layer_output_derivatives @ second_term

        # Update the weights using the output derivative calculated above
        for curr_layer in layers_reversed:

            # Get the activation for this layer and its derivative function
            activation_for_this_layer = self.activations[curr_layer]
            activation_derivative = self.activations_derivatives[activation_for_this_layer]

            # If first layer then use the data as the previous layer.
            if curr_layer == 0:
                previous_layer_output = x
            else:
                prev_layer = curr_layer - 1
                previous_layer_output = out[prev_layer].copy()
                previous_layer_output = np.insert(previous_layer_output, obj = 0, values = 1)

            # Current layer output derivatives
            curr_layer_output_derivatives = output_derivatives[curr_layer].reshape(-1, 1)

            # Get current layer's activation derivatives
            curr_layer_activation_derivatives = activation_derivative(self.old_weights[curr_layer] @ previous_layer_output)
            curr_layer_activation_derivatives = curr_layer_activation_derivatives.reshape(-1, 1)

            # For the current layer multiply each neuron's activation derivatives with all previous layer outputs.
            curr_layer_weight_derivatives = curr_layer_output_derivatives * \
                                            curr_layer_activation_derivatives * previous_layer_output
            weight_derivatives[curr_layer] = curr_layer_weight_derivatives

        # Append the current data point's weight derivatives in the batch derivatives array
        self.batch_weight_derivatives.append(weight_derivatives)

    def _forward(self, x):
        out = []
        for curr_layer in self.layers:
            out.append([])

            # Get the activation for this layer and its function
            activation_for_this_layer = self.activations[curr_layer]
            activation_function = self.activations_functions[activation_for_this_layer]

            if curr_layer == 0:
                previous_layer_output = x
            else:
                previous_layer_output = out[curr_layer - 1].copy()
                previous_layer_output = np.insert(previous_layer_output, obj = 0, values = 1)

            out[curr_layer] = activation_function(self.weights[curr_layer] @ previous_layer_output)

        out = np.array(out)
        return out

    def fit(self, X, y):

        # Add a bias column to X
        X_new = np.column_stack((np.ones(len(X)), X))

        # Initialise the weights of the network
        self._initialise_weights(X_new.shape[1])

        for epoch in range(self.epochs):

            # Initialise arrays to store all weight derivatives of the batch
            self.batch_weight_derivatives = []

            # Update weights using mini-batch stochastic gradient descent
            for data_index in range(X_new.shape[0]):
                out = self._forward(X_new[data_index])
                self._backward(X_new[data_index], y[data_index], out)

                if (data_index + 1) % self.batch_size == 0:
                    self._update_weights()

            predictions = self.predict(X)
            loss = self._mse_loss(predictions, y)
            print("Epoch = ", str(epoch + 1), " - ", "Loss = ", str(loss))

    def predict(self, X):

        # Add a bias column to X
        X_new = np.column_stack((np.ones(len(X)), X))

        preds = []
        for x in X_new:
            pred = self._forward(x)[-1]
            preds.append(pred)

        preds = np.array(preds).flatten()
        return preds

## Autoencoder

## KMeans

## MICE

## EM